## Benchmark numerical 1D minimization

$\max_{\lambda\in[0, \frac{1}{B-\mu}]} \mathbb{E}_{X\sim\nu}[\log(1-(X-\mu)\lambda)]$

where $\nu=\frac{1}{n}\sum_{i=1}^n \delta_{x_i}$ is an empirical measure and $\max_{i=1, \dots, n} x_i \leq B$.

In [1]:
import numpy as np
from scipy.optimize import minimize, minimize_scalar, root_scalar

In [2]:
mu = 0.51
upper_bound = 1.0
n = 500
X = upper_bound * np.random.rand(n)

In [3]:
def test_trust_constr(n, X, upper_bound, mu):
    def f(l):
        return -np.mean(np.log(1 - (X - mu) * l))

    def jac(l):
        return np.mean((X - mu) / (1 - (X - mu) * l))

    def hess(l):
        return np.mean((X - mu) ** 2 / (1 - (X - mu) * l) ** 2)

    ret = minimize(
        f, 0.5 / (upper_bound - mu), method='trust-constr', jac=jac, hess=hess, bounds=[(0, 1 / (upper_bound - mu))]
    )
    
    if ret.success:
        return ret.fun
    else:
        return None
    
def test_trust_constr_no_hess(n, X, upper_bound, mu):
    def f(l):
        return -np.mean(np.log(1 - (X - mu) * l))

    def jac(l):
        return np.mean((X - mu) / (1 - (X - mu) * l))

    ret = minimize(
        f, 0.5 / (upper_bound - mu), method='trust-constr', jac=jac, bounds=[(0, 1 / (upper_bound - mu))]
    )
    
    if ret.success:
        return ret.fun
    else:
        return None
    
def test_trust_constr_no_jac(n, X, upper_bound, mu):
    def f(l):
        return -np.mean(np.log(1 - (X - mu) * l))

    ret = minimize(
        f, 0.5 / (upper_bound - mu), method='trust-constr', bounds=[(0, 1 / (upper_bound - mu))]
    )
    
    if ret.success:
        return ret.fun
    else:
        return None

In [4]:
print('minimize, trust-constr')
%timeit test_trust_constr(n, X, upper_bound, mu)
print(test_trust_constr(n, X, upper_bound, mu))

print('minimize, trust-constr, no hess')
%timeit test_trust_constr_no_hess(n, X, upper_bound, mu)
print(test_trust_constr_no_hess(n, X, upper_bound, mu))

print('minimize, trust-constr, no jac')
%timeit test_trust_constr_no_jac(n, X, upper_bound, mu)
print(test_trust_constr_no_jac(n, X, upper_bound, mu))

minimize, trust-constr
71.1 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
-0.0009623123891665784
minimize, trust-constr, no hess
77.6 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
-0.00096231238885155
minimize, trust-constr, no jac
78.4 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
-0.000962312388888497


In [5]:
def test_minimize_scalar_bounded(n, X, upper_bound, mu):
    def f(l):
        return -np.mean(np.log(1 - (X - mu) * l))

    ret = minimize_scalar(
        f, method='bounded', bounds=(0, 1 / (upper_bound - mu))
    )
    
    if ret.success:
        return ret.fun
    else:
        return None

In [6]:
print('minimize_scalar, bounded')
%timeit test_minimize_scalar_bounded(n, X, upper_bound, mu)
print(test_minimize_scalar_bounded(n, X, upper_bound, mu))

minimize_scalar, bounded
850 µs ± 43.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-0.0009623127569702782


In [9]:
def test_root_scalar_brentq(n, X, upper_bound, mu):
    def f(l):
        return np.mean(np.log(1 - (X - mu) * l))
    def jac(l):
        return -np.mean((X - mu) / (1 - (X - mu) * l))
    
    l_plus = 1 / (upper_bound - mu)
    
    if jac(0) * jac(l_plus) >= 0:
        return np.maximum(f(0), f(l_plus))
    else:
        ret = root_scalar(
            jac, method='brentq', bracket=[0, l_plus]
        )
        if ret.converged:
            return np.max([f(ret.root), f(0), f(l_plus)])
        else:
            return None

In [10]:
print('root_scalar, brentq')
%timeit test_root_scalar_brentq(n, X, upper_bound, mu)
print(test_root_scalar_brentq(n, X, upper_bound, mu))

root_scalar, brentq
336 µs ± 29.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.000962312756970942
